# NKSOPS_03_DataTransformation_02

In [ ]:
#load Enviornment
%run NKSOPS_02_ClassAndMethods_02.ipynb

In [ ]:
# Get count of tables
# file = pd.read_html(r"C:\Users\76350\Documents\NKS_OPS\03_Data\htmlfiles\502 Mx Os.htm")
# print(len(file.index))

In [ ]:
count = 0
mergedtables = pd.DataFrame()
while count <= 12:
    filecurr = pd.read_html(r"C:\Users\76350\Documents\NKS_OPS\03_Data\htmlfiles\502 Mx Os.htm")[count]
    count = count + 1
    display(filecurr)

In [ ]:
#Assign each table to a variable
event_chronology = pd.read_html(r"C:\Users\76350\Documents\NKS_OPS\03_Data\htmlfiles\502 Mx Os.htm")[9]
troubleshooting = pd.read_html(r"C:\Users\76350\Documents\NKS_OPS\03_Data\htmlfiles\502 Mx Os.htm")[10]

#Rename columns and create unique ID
troubleshooting.columns= ['TS_Action','TS_Department','TS_Next_Department','TS_Description']
troubleshooting = troubleshooting.rename_axis('ID').reset_index()
troubleshooting['EC_TS_ID'] = troubleshooting['ID'].astype(str) +"_" +  troubleshooting['TS_Action']

#Rename columns and create unique ID
event_chronology.columns= ['Event_Chronology','EC_Action','EC_Date','EC_Airport','EC_Category']
event_chronology = event_chronology.rename_axis('ID').reset_index()
event_chronology['ID_EC_TS'] = event_chronology['ID'].astype(str) +"_" +  event_chronology['EC_Category']

event_chronology_rename = event_chronology[event_chronology['EC_Category'] == 'Information'].reset_index().rename_axis('ID_EC').reset_index()
event_chronology_rename['EC_TS_ID'] = event_chronology_rename['ID_EC'].astype(str) +"_" +  event_chronology_rename['EC_Category']

#Merge the two tables on the unique ID and event order of information
merged_ec_ts = event_chronology_rename.merge(troubleshooting, on="EC_TS_ID").drop(columns =['ID_EC','ID_x','ID_y'])
merged_ec_ts['ID_EC_TS'] = merged_ec_ts['index'].astype(str) +"_" +  merged_ec_ts['TS_Action']
merged_ec_ts = event_chronology.merge(merged_ec_ts, on="ID_EC_TS", how = "left")

display(merged_ec_ts)


In [ ]:
# Merge all other tables that are not event_chronology and troubleshooting
count = 0
mergedtables = pd.DataFrame()
while count <= 9:
    filecurr = pd.read_html(r"C:\Users\76350\Documents\NKS_OPS\03_Data\htmlfiles\502 Mx Os.htm")[count]
    mergedtables = pd.concat([mergedtables, filecurr.reindex(filecurr.index)], axis=1)
    count = count + 1
    
count2 = 11
mergedtables = mergedtables
while count2 <= 12:
    filecurr2 = pd.read_html(r"C:\Users\76350\Documents\NKS_OPS\03_Data\htmlfiles\502 Mx Os.htm")[count2]
    mergedtables = pd.concat([mergedtables, filecurr2.reindex(filecurr2.index)], axis=1)
    count2 = count2 + 1

    
mergedtables =  mergedtables.rename(columns = {"Date/Time (Lt)": 'Date_Time',"Log Page:":'Log_Page'}).fillna(method='ffill')
# mergedtables["Unique_Flight"] = mergedtables["Date_Time"].astype(object) + "_" + mergedtables["Tail"].astype(str)
# mergedtables[['Date','Time']] = mergedtables['Date_Time'].astype(object).str.split(' ',expand=True)


mergedtablesfin = pd.concat([mergedtables, merged_ec_ts.reindex(merged_ec_ts.index)], axis=1)

display(list(mergedtablesfin))
display(mergedtablesfin)
mergedtablesfin.to_csv(r"C:\Users\76350\Documents\NKS_OPS\03_Data\htmlfiles\mergedtables.csv")